In [1]:
# This is file for train, prediction

import pandas as pd

dirpath = 'C:/Python/Used-Car-Price-Regression-DACON/'

train = pd.read_csv('data/modified_train.csv')
test = pd.read_csv('data/modified_test.csv')

In [2]:
from sklearn import model_selection

# train data for scaler fitting

Y = train[ ['target'] ].values
X = train[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values
X_test = test[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values

In [3]:
from sklearn.preprocessing import MinMaxScaler

scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)
X_test = scalerX.transform(X_test)

scalerY = MinMaxScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)

In [4]:
x_train, x_val, y_train, y_val = \
    model_selection.train_test_split(X, Y, test_size=0.3, random_state=41)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((710, 10), (305, 10), (710, 1), (305, 1))

In [5]:
from pycaret import regression

model = regression.setup(data=train, target='target',train_size=0.7,session_id=0,silent=True)

,Description,Value
0,session_id,0
1,Target,target
2,Original Data,"(1015, 11)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(710, 38)"


In [6]:
top_3_models = regression.compare_models(sort='MSE', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,2798996.0519,69978546911409.4062,7259353.9178,0.6455,0.5150,0.4127,0.3610
gbr,Gradient Boosting Regressor,3293840.1210,79532199490951.0781,7993333.3380,0.5670,0.5894,0.5241,0.0210
lightgbm,Light Gradient Boosting Machine,3462953.7833,81667255508237.7812,8042259.7623,0.5595,0.6361,0.6030,0.0140
rf,Random Forest Regressor,3145821.1175,83276610198918.2031,8071626.6693,0.5610,0.4672,0.3999,0.0740
xgboost,Extreme Gradient Boosting,2892771.3516,85642911899571.4844,7980263.1722,0.5774,0.5035,0.3999,0.0450
et,Extra Trees Regressor,3506710.3085,94957637748232.1094,9057354.0706,0.4168,0.5301,0.5090,0.0640
ridge,Ridge Regression,5420395.5549,112043152608236.4688,9894490.5393,0.3073,0.8702,1.1912,0.0040
lr,Linear Regression,5443164.0507,112189838493328.8750,9891862.8761,0.3091,0.8871,1.1960,0.3040
llar,Lasso Least Angle Regression,5478062.1710,112736502659544.8438,9930314.1667,0.3002,0.8773,1.2073,0.0050
lasso,Lasso Regression,5476413.2901,112737947581770.9219,9930397.6464,0.3002,0.8771,1.2068,0.0050


In [7]:
blended = regression.blend_models(estimator_list=top_3_models, fold=10, optimize='MSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2396021.6348,18949291879001.8320,4353078.4370,0.8291,0.4767,0.5287
1,4504312.4419,318473560602811.6875,17845827.5404,0.2027,0.6464,0.4686
2,2668800.1134,25649624628970.3750,5064545.8463,0.7901,0.5335,0.5733
3,2951457.9609,106883263532132.5781,10338436.2228,0.5381,0.5157,0.5140
4,2011823.1888,10556913153757.5859,3249140.3715,0.8201,0.4773,0.3761
5,2590071.7061,23849939355906.1680,4883639.9699,0.7093,0.5342,0.5578
6,4174644.8730,120743160632755.0156,10988319.2815,0.5663,0.5864,0.4732
7,3255292.7595,40464074269004.0234,6361137.8125,0.4556,0.4265,0.3657
8,2269903.7283,23479922747074.1719,4845608.6044,0.6419,0.5075,0.4330


In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error

from itertools import combinations
from vecstack import StackingTransformer
from tqdm import tqdm

def NMAE(true, pred):
    mae = mean_absolute_error(true,pred)
    score = mae / np.mean(np.abs(true))
    return score

In [9]:
print(X.shape, X_test.shape, Y.shape)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(1015, 10) (436, 10) (1015, 1)
(710, 10) (305, 10) (710, 1) (305, 1)


In [10]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = [
    ('LGBMRegressor',LGBMRegressor(n_estimators=134,max_depth=16,random_state=41)),
    ('XGBRegressor',XGBRegressor(n_estimators=194,max_depth=7,random_state=41,verbosity=0)),
    ('CatBoostRegressor',CatBoostRegressor(n_estimators=1200,max_depth=8,random_state=41,verbose=0)),
    ('GradientBoostingRegressor',GradientBoostingRegressor(n_estimators=1301,max_depth=9,learning_rate=0.01,subsample=0.5,random_state=41)),
    ('ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=344,max_depth=16,random_state=41,criterion='mse')),
    ('RandomForestRegressor',RandomForestRegressor(n_estimators=177,max_depth=16,random_state=41,criterion='mse'))]

In [11]:
for name, model in models:
    model.fit(x_train, y_train)
    print(f'{name}: ', NMAE(y_val,model.predict(x_val)))

LGBMRegressor:  0.4823591596046905
XGBRegressor:  0.3459332187495732
CatBoostRegressor:  0.33229571077726616
GradientBoostingRegressor:  0.37056434476663025
ExtraTreesRegressor:  0.35692932414699813
RandomForestRegressor:  0.3625769917003506


In [12]:
pred_list = list()
for name, model in models:
    pred_list.append([model.predict(x_val),name])

pred_comb = list()
for i in range(2,len(pred_list)+1):
    pred_comb += list(combinations(pred_list, i))

In [13]:
nmae_list = list()

for pred_info in pred_comb:
    preds, names = list(), list()

    for pred, name in pred_info:
        preds.append(pred)
        names.append(name)
    nmae_list.append([NMAE(y_val,sum(preds)/len(preds)),names])

In [14]:
sorted(nmae_list)[0]

[0.325187047550314,
 ['XGBRegressor', 'CatBoostRegressor', 'ExtraTreesRegressor']]

In [15]:
estimator = [('XGBRegressor', XGBRegressor(n_estimators=194, max_depth=7, random_state=41, verbosity=0)),
('CatBoostRegressor', CatBoostRegressor(n_estimators=1200, max_depth=8, random_state=41, verbose=0)),
('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=344, max_depth=16, random_state=41, criterion='mse'))]

In [16]:
stack = StackingTransformer(estimator,
                            regression = True,
                            metric = NMAE,
                            n_folds = 10, stratified = True, shuffle = True,
                            random_state = 41, verbose = 0)

In [17]:
S_train = stack.fit_transform(x_train, y_train)
S_test = stack.transform(x_val)

S_model = LGBMRegressor(n_estimators=100,max_depth=2,random_state=41)
S_model.fit(S_train, y_train)
NMAE(y_val, S_model.predict(S_test))

0.38343756538531665

In [18]:
models = [
    ('XGBRegressor', XGBRegressor(n_estimators=194, max_depth=7, random_state=41, verbosity=0)),
    ('CatBoostRegressor', CatBoostRegressor(n_estimators=1200, max_depth=8, random_state=41, verbose=0)),
    ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=344, max_depth=16, random_state=41, criterion='mse'))]

In [19]:
pred = None
for name, model in models:
    model.fit(x_train, y_train)
    if pred is None:
        pred = model.predict(X_test)
    else:
        pred += model.predict(X_test)

pred = pred/len(models)
pred = scalerY.inverse_transform([pred])

In [20]:
pred = pred.reshape(-1,1)

In [21]:
pred.shape

(436, 1)

In [22]:
sample = pd.read_csv('sample_submission.csv')
sample['target'] = pred

sample.to_csv( dirpath + "submit.csv", index=False)